## Imports

In [ ]:
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.models import Simulation, SimulationCampaign, SimulationResult, SimulationExecution
from obi_auth import get_token
from obi_notebook.get_projects import get_projects

## Get authentication token and choose a project context

In [ ]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects(token=token)

## Initialize the client with the token and project context

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

## Select SimulationCampaigns
Either by:
1. Specifying SimulationCampaign IDs
2. Selecting from the widget

In [ ]:
from obi_notebook import get_entities

# Optional: Download using unique ID
entity_IDs = "<SIMULATION-CAMPAIGN-IDs>"  # <<< FILL IN A LIST OF UNIQUE SIMULATION CAMPAIGN IDENTIFIERS HERE


if entity_IDs != "<SIMULATION-CAMPAIGN-IDs>":
    simulation_campaign_ids = [entity_IDs]
else:
# Alternative: Select from a table of entities
    simulation_campaign_ids = []
    simulation_campaign_ids = get_entities.get_entities("simulation-campaign", token, simulation_campaign_ids,
                                          env="production",
                                          project_context=project_context,
                                          multi_select=True,
                                          page_size=100)

## Print the selected IDs
If you used the widget you might want to copy these the entity_IDs parameter so they are already selected for next time

In [ ]:
print(simulation_campaign_ids)

## Helper function for downloading a SimulationCampaign

In [ ]:
from entitysdk.staging.simulation import stage_simulation
from entitysdk.staging.simulation_result import stage_simulation_result

def stage_sim_campaign(simulation_campaign_id, output_dir):
    simulation_campaign = client.get_entity(entity_type=SimulationCampaign, entity_id=simulation_campaign_id)

    simulation_config_paths = []

    for id, simulation in enumerate(simulation_campaign.simulations):
        
        simulation_executions = client.search_entity(
                    entity_type=SimulationExecution,
                    query={"used__id": simulation.id}
                ).all()
        
        if len(simulation_executions) == 0:
            print(f"Warning: No SimulationExecution found for Simulation ID: {simulation.id}")
        simulation_execution = simulation_executions[0]
        
        simulation_result_id = simulation_execution.generated[0].id
        simulation_result = client.get_entity(entity_type=SimulationResult, entity_id=simulation_result_id)
        
        simulation_config_path = stage_simulation_result(
            client=client,
            model=simulation_result,
            output_dir=output_dir + f"/simulation_{id}",
            simulation_config_file=None,
        )
        simulation_config_paths.append(simulation_config_path)

    return simulation_config_paths

# Stage each SimulationCampaign

In [ ]:
simulation_config_paths_by_campaign = []
for i, simulation_campaign_id in enumerate(simulation_campaign_ids):
    print(f"Staging simulation campaign {i+1}/{len(simulation_campaign_ids)}: {simulation_campaign_id}")

    output_dir = f"./simulation_results/{simulation_campaign_id}"
    simulation_config_paths = stage_sim_campaign(simulation_campaign_id, output_dir=output_dir)
    simulation_config_paths_by_campaign.append(simulation_config_paths)

# Plot and print the spikes

In [ ]:
for simulation_config_paths in simulation_config_paths_by_campaign:
    for simulation_config_path in simulation_config_paths:
        print(f"Processing simulation config path: {simulation_config_path}")
        
        # Plot and print the spikes
        import bluepysnap
        snap_simulation = bluepysnap.Simulation(simulation_config_path)
        spikes = snap_simulation.spikes
        spikes.filter().raster()

        print(spikes.population_names)
        for population in spikes.population_names:
            population_spikes = spikes[population]
            print(population_spikes.get())